In [1]:
import pandas as pd
from ast import literal_eval
import json

In [2]:
df = pd.read_csv('QASet.csv').fillna('null')

In [3]:
df

,stage,id,sub_id,question,judgement,conflict,jump
0,1,1,1.1.1,年齡,null,null,null
1,1,2,1.2.1,性別,null,null,null
2,1,3,1.3.1,情感狀況,null,null,null
3,2,1,2.1.1,通勤,null,null,null
4,2,2,2.2.1,外食or家裡,null,null,"[{""answer"":[""外食"",""家裡""],""jump"":[""2.3.1"",""2.4.1""]}]"
5,2,3,2.3.1,餐館or夜市or超商or公司,null,null,3.1.1
6,2,4,2.4.1,家裡吃得豐盛or簡單,"[{""answer"":[""豐盛"",""簡單""],""rank"":[1,2]}]",null,3.1.1
7,3,1,3.1.1,覺得如何,null,null,null


In [4]:
def ruleParser(str):
    lst = literal_eval(str)
    rule_df = pd.DataFrame(lst[0])
    return rule_df

In [5]:
jdf = ruleParser(df.iloc[6]['judgement'])

In [6]:
jdf

,answer,rank
0,豐盛,1
1,簡單,2


In [7]:
answer = '豐盛'
rank = jdf[jdf.answer==answer].loc[0]['rank']

In [8]:
rank

1

In [10]:
index = 0
while index < len(df):
    question = df.iloc[index]['question']
    print(question)
    #之後可以做一個專門的user_answer_analyzer，這樣就可以回傳被「正規化」的答案
    user_answer = input()
    #條件判斷
    if df.iloc[index]['judgement'] != 'null':
        print(ruleParser(df.iloc[index]['judgement']))
    else:
        pass
    #跳轉判斷
    if df.iloc[index]['jump'] != 'null':
        try:
            rule = ruleParser(df.iloc[index]['jump'])
            print(rule)
            jump_to = rule[rule.answer == user_answer].iloc[0]['jump']
            print(df[df.sub_id == jump_to].index.to_list()[0])
            print(index)
            distance = df[df.sub_id == jump_to].index.to_list()[0] - index
            index = index + distance
        except:
            jump_to = df.iloc[index]['jump']
            distance = df[df.sub_id == jump_to].index.to_list()[0] - index
            index = index + distance
    else:
        index = index + 1

年齡
21
性別
男
情感狀況
solo
通勤
bus
外食or家裡
家裡
  answer   jump
0     外食  2.3.1
1     家裡  2.4.1
6
4
家裡吃得豐盛or簡單
簡單
  answer  rank
0     豐盛     1
1     簡單     2
覺得如何
還行
